In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report

# =====================================================
# 1️⃣ 데이터 로드 + 시간 파생
# =====================================================
df = pd.read_csv("new_flight_weather_merged.csv")

df["departure_datetime"] = pd.to_datetime(df["departure_datetime"])
df["dep_hour"] = df["departure_datetime"].dt.hour
df["dep_weekday"] = df["departure_datetime"].dt.weekday
df["is_weekend"] = df["dep_weekday"].isin([5, 6]).astype(int)

# =====================================================
# 2️⃣ 상태 → 다중분류 라벨
# =====================================================
label_map = {
    "정상운항": "문제없음",
    "지연": "지연",
    "회항": "회항",
    "취소": "취소"
}

df = df[df["상태"].isin(label_map.keys())].copy()
df["target"] = df["상태"].map(label_map)

print("📊 원본 클래스 분포")
print(df["target"].value_counts())

# =====================================================
# 3️⃣ 컬럼 정의
# =====================================================
num_cols = ["기온(°C)", "풍속_ms", "dep_hour", "dep_weekday", "is_weekend"]
num_cols = [c for c in num_cols if c in df.columns]

cat_cols = ["공항명", "출발지", "도착지", "flight_type"]
cat_cols = [c for c in cat_cols if c in df.columns]

for c in cat_cols:
    df[c] = df[c].astype("category")

X_cols = num_cols + cat_cols

# =====================================================
# 4️⃣ Train / Test 분리 (시간 기준)
# =====================================================
df = df.sort_values("departure_datetime")
split_date = df["departure_datetime"].quantile(0.8)

train_df = df[df["departure_datetime"] <= split_date]
test_df  = df[df["departure_datetime"] > split_date]

print("\nTrain:", len(train_df), "Test:", len(test_df))

# =====================================================
# 5️⃣ 🔥 취소 기준 다운사이징 (⚠️ 실험용)
# =====================================================
train_ok     = train_df[train_df["target"] == "문제없음"]
train_delay  = train_df[train_df["target"] == "지연"]
train_divert = train_df[train_df["target"] == "회항"]
train_cancel = train_df[train_df["target"] == "취소"]

base_n = len(train_cancel)  # 🔥 취소 기준

print("\n기준 샘플 수 (취소):", base_n)

train_ok_down = train_ok.sample(n=base_n, random_state=42)
train_delay_down = train_delay.sample(n=base_n, random_state=42)

# 회항은 부족하면 그대로 사용
train_divert_down = train_divert if len(train_divert) <= base_n else \
    train_divert.sample(n=base_n, random_state=42)

train_down = pd.concat([
    train_ok_down,
    train_delay_down,
    train_divert_down,
    train_cancel
]).sample(frac=1, random_state=42)

print("\n📊 다운사이징 후 Train 분포")
print(train_down["target"].value_counts())

# =====================================================
# 6️⃣ X / y 분리
# =====================================================
X_train = train_down[X_cols]
y_train = train_down["target"]

X_test  = test_df[X_cols]
y_test  = test_df["target"]

# =====================================================
# 7️⃣ LightGBM 다중분류
# =====================================================
lgbm = LGBMClassifier(
    objective="multiclass",
    num_class=4,
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

lgbm.fit(
    X_train,
    y_train,
    categorical_feature=cat_cols
)

print("\n✅ LightGBM (취소 기준 다운사이징) 학습 완료")

# =====================================================
# 8️⃣ 평가
# =====================================================
y_pred = lgbm.predict(X_test)

print("\n📊 Classification Report (실험용)")
print(classification_report(y_test, y_pred))


C:\Users\Admin\AppData\Local\Temp\ipykernel_13220\3103664467.py:9: DtypeWarning: Columns (28,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("new_flight_weather_merged.csv")


📊 원본 클래스 분포
문제없음    2360824
지연       481449
회항          859
취소          579
Name: target, dtype: int64

Train: 2274969 Test: 568742

기준 샘플 수 (취소): 464

📊 다운사이징 후 Train 분포
문제없음    464
회항      464
취소      464
지연      464
Name: target, dtype: int64
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000142 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 466
[LightGBM] [Info] Number of data points in the train set: 1856, number of used features: 9
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits